**Objective** Insurance companies are using Random Forest algoritham to automatically approve or flag cancer-related insurance claims using real-world-like data (Here I used such as from Kaggle)
By Using Supervised machine learning algorithm(Random Forest) A decision is made:
Approve a cancer insurance claim
Reject or flag it for manual review
Why random Forest algoritham is taken -Random Forest is a strong and often exclusive choice in insurance claim prediction:
Handles mixed data types
Resistant to overfitting
No need for scaling
Handles missing data
Feature importance
Performs well on imbalanced data
Non-linear modeling

In [2]:
# Install Kaggle CLI
!pip install kaggle

In [4]:
# Move the JSON File and Set Permissions**
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#Install the Kaggle CLI
!pip install -q kaggle

In [7]:
!kaggle datasets download -d uciml/breast-cancer-wisconsin-data

Dataset URL: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data
License(s): CC-BY-NC-SA-4.0


In [8]:
!ls


breast-cancer-wisconsin-data.zip  kaggle.json  sample_data


In [9]:
!unzip breast-cancer-wisconsin-data.zip

Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


In [10]:
!ls

breast-cancer-wisconsin-data.zip  data.csv  kaggle.json  sample_data


In [11]:
# Load with pandas
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  texture_worst  perimeter_worst  area_worst  smoothness_worst  \
0  ...          17.33           184.60      2019.0            0.1622   
1  ...          23.41           158.80      1956.0            0.1238   
2  ...          25.53           152.50      1709.0            0.1444   
3  ...          26.50            98.87       567.7            0.2098   
4  ...          16.67           152.20      1575.0            0.1374   

   compactness_worst  concavity_worst  concave points_worst  symmetry_worst  \
0             0.6656           0.7119                0.2654          0.4601   
1             0.1866           0.2416                0.1860          0.2750   
2             0.4245           0.4504                0.2430          0.3613   
3             0.8663           0.6869                0.2575          0.6638   
4             0.2050           0.4000                0.1625          0.2364   

   fractal_dimension_worst  Unnamed: 32  
0                  0.11890          NaN  
1                  0.08902          NaN  
2                  0.08758          NaN  
3                  0.17300          NaN  
4                  0.07678          NaN  

[5 rows x 33 columns]

The above dataset is having Columns that include:
Patient ID
Diagnosis (M = malignant, B = benign)
Tumor features (e.g., radius_mean, texture_mean, etc.)
Age, policy details
Claim amount
Claim outcome (approved/rejected)

We are simulating claim status using diagnosis + claim size logic with below logic
Model: Random Forest Classifier
Input: Diagnosis + tumor features + policy type
Output: Whether to approve or reject a claim

In [19]:
#Importing some libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
# Convert diagnosis to binary
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

In [14]:
# Simulate a "claim_approved" column
# Rule: if malignant and radius_mean > 15 → more likely to be approved
df['claim_approved'] = ((df['diagnosis'] == 1) & (df['radius_mean'] > 15)).astype(int)

In [15]:
# Features and target
X = df[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'diagnosis']]
y = df['claim_approved']

In [16]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)


In [17]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [18]:
# Evaluate
y_pred = rf.predict(X_test)
print("📊 Claim Approval Classification Report:")
print(classification_report(y_test, y_pred))

📊 Claim Approval Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       103
           1       1.00      0.97      0.99        40

    accuracy                           0.99       143
   macro avg       1.00      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143



In [20]:
# Count how many claims are predicted as approved (1)
approved_count = np.sum(y_pred == 1)
rejected_count = np.sum(y_pred == 0)

print(f"✅ Approved claims: {approved_count}")
print(f"❌ Rejected claims: {rejected_count}")

✅ Approved claims: 39
❌ Rejected claims: 104


In [21]:
total = len(y_pred)
print(f"✅ Approved (%): {approved_count / total * 100:.2f}%")
print(f"❌ Rejected (%): {rejected_count / total * 100:.2f}%")

✅ Approved (%): 27.27%
❌ Rejected (%): 72.73%


In [22]:
# Confusion Matrix for Full Comparison

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("🔢 Confusion Matrix:")
print(cm)

🔢 Confusion Matrix:
[[103   0]
 [  1  39]]


In above we counted the number of predicted approvals in y_pred.Assuming 1 means "approved" and 0 means "rejected".Please be aware that Insurance firms will also use medical notes (NLP), policy rules, fraud indicators, and expert review before taking any decison.ML outputs are often used to assist, not replace, manual decisions in healthcare.